In [353]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import datetime
import time
import csv

In [354]:
def tableText(table):    
    """
    Returns a list of column rows and a header if applicable. 
    """
    def getRowText(tr, coltag='td'):     
        return [td.get_text(strip=True) for td in tr.find_all(coltag)]  
    rowsList = []
    tableRows = table.find_all('tr')
    tableHeader = getRowText(tableRows[0], 'th')
    if tableHeader:
        rowsList.append(tableHeader)
        tableRows = tableRows[1:]
    for i in tableRows:
        rowsList.append(getRowText(i, 'td') )       
    return rowsList

In [355]:
def date_conversion(date):
     return datetime.datetime.strptime(date, ' %b %d, %Y')

In [356]:
def addTeams(df):
    i = 0
    q = int(df[df['Name']== 'Name'].index.values) + 1

    while i < len(df)-1:
        if i == (q-2):
            i += 1
        elif i == (q-1):
            df.loc[(q-2), 'Team'] = teamsList[0]
            i += 1
        elif int(df[df['Name'] == df.Name[i]].index.values) < (q-1):
            mask = int(df[df['Name'] == df.Name[i]].index.values)
            df.loc[mask, 'Team'] = teamsList[0]
            i += 1
        else:
            mask = int(df[df['Name'] == df.Name[i]].index.values)
            df.loc[mask, 'Team'] = teamsList[1]
            i += 1  

In [357]:
def StatsTable(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    table = soup.find(lambda tag: tag.name=='table' and tag.has_attr('id') and tag['id']=="roster") 
    rows = table.findAll(lambda tag: tag.name=='tr')[1]
    teams = table.findAll(lambda tag: tag.name=='tr')[0]
    
    teamsList = [team.text for team in teams.findAll(lambda tag: tag.name=='td')]
    list_table = tableText(rows)
        
    dftable = pd.DataFrame(list_table[1:], columns=list_table[0])
    dftable['Gameday'] = soup.title.string
        
    split1 = dftable['Gameday'].str.split(' - ', n = 1, expand = True)
    dftable['Date'] = split1[1]
        
    split2 = split1[0].str.split(' at ', n = 1, expand = True)
    dftable['Location'] = split2[1]
    dftable.drop(columns = ['Gameday'], inplace = True)
        
    split3 = dftable['Date'].str.split(',', n = 1, expand = True)
    dftable['Day'] = split3[0]
    dftable['Date'] = split3[1]
    dftable['Date'] = dftable['Date'].apply(date_conversion)
    
    q = int(dftable[dftable['Name']== 'Name'].index.values) + 1

    for i in range(0,(len(dftable)-1)):
        if i == q-2:
            continue
        else: 
            mask = int(dftable[(dftable['Name'] == dftable.Name[i]) & 
                               (dftable['#'] == dftable['#'][i])
                              & (dftable['LB'] == dftable['LB'][i])].index.values)

            if mask < (q-1):
                dftable.loc[mask, 'Team'] = teamsList[0]
            elif mask > (q-1):
                dftable.loc[mask, 'Team'] = teamsList[1]
            
    return dftable

In [358]:
url = 'https://laxreports.sportlogiq.com/nll/GS179.html'
# url1 = 'https://laxreports.sportlogiq.com/nll/GS2276.html'
# url2 = 'https://laxreports.sportlogiq.com/nll/GS800.html'
# url3 = 'https://laxreports.sportlogiq.com/nll/GS900.html'
# url4 = 'https://laxreports.sportlogiq.com/nll/GS810.html'

In [359]:
dftable1 = StatsTable(url)
df_list = [dftable1]
dftable1.head()

,#,Name,G,A,+/-,PIM,S,SOFF,LB,T,CT,FO,TOF,Date,Location,Day,Team
0,0,T.Welsh,2,0,0,4,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
1,0,P.Welsh,0,0,0,2,0,0,5,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
2,0,J.Tucker,2,0,0,2,0,0,6,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
3,0,R.Sowell,0,0,0,0,0,0,0,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
4,0,D.Smith,0,0,0,0,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster


In [360]:
def StatsTableUpdate(urlsList):
    for url in urlsList:
        # print(url)
        dftablei = StatsTable(url)
        # print(dftablei.head(3))
        df_list.append(dftablei)
        time.sleep(1)

Test if having issues with any sites

In [361]:
# dftable2 = StatsTable('https://laxreports.sportlogiq.com/nll/GS920.html')
# dftable2.head()

In [362]:
urlsList = []

for i in range(236,2277):
    url = 'https://laxreports.sportlogiq.com/nll/GS{}.html'.format(i)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    if bool(soup.findAll(text="html")):
        urlsList.append(url)

urlsList

['https://laxreports.sportlogiq.com/nll/GS236.html',
 'https://laxreports.sportlogiq.com/nll/GS305.html',
 'https://laxreports.sportlogiq.com/nll/GS310.html',
 'https://laxreports.sportlogiq.com/nll/GS360.html',
 'https://laxreports.sportlogiq.com/nll/GS374.html',
 'https://laxreports.sportlogiq.com/nll/GS389.html',
 'https://laxreports.sportlogiq.com/nll/GS391.html',
 'https://laxreports.sportlogiq.com/nll/GS393.html',
 'https://laxreports.sportlogiq.com/nll/GS394.html',
 'https://laxreports.sportlogiq.com/nll/GS412.html',
 'https://laxreports.sportlogiq.com/nll/GS416.html',
 'https://laxreports.sportlogiq.com/nll/GS417.html',
 'https://laxreports.sportlogiq.com/nll/GS419.html',
 'https://laxreports.sportlogiq.com/nll/GS420.html',
 'https://laxreports.sportlogiq.com/nll/GS426.html',
 'https://laxreports.sportlogiq.com/nll/GS430.html',
 'https://laxreports.sportlogiq.com/nll/GS435.html',
 'https://laxreports.sportlogiq.com/nll/GS445.html',
 'https://laxreports.sportlogiq.com/nll/GS448.

In [363]:
StatsTableUpdate(urlsList)

In [364]:
urlsList800 = urlsList

In [365]:
fulldf = pd.concat(df_list, ignore_index=True)
fulldf

,#,Name,G,A,+/-,PIM,S,SOFF,LB,T,CT,FO,TOF,Date,Location,Day,Team
0,0,T.Welsh,2,0,0,4,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
1,0,P.Welsh,0,0,0,2,0,0,5,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
2,0,J.Tucker,2,0,0,2,0,0,6,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
3,0,R.Sowell,0,0,0,0,0,0,0,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
4,0,D.Smith,0,0,0,0,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59619,86,J.Lintz,0,0,-1,2,0,0,1,0,0,-,16:21,2020-03-08,Colorado,Sun,Colorado Roster
59620,91,J.McNabb,0,2,4,0,3,4,2,2,1,-,11:03,2020-03-08,Colorado,Sun,Colorado Roster
59621,30,T.Carlson,0,0,,0,0,0,3,1,0,-,,2020-03-08,Colorado,Sun,Colorado Roster
59622,45,D.Ward,0,0,,0,0,0,1,0,0,-,,2020-03-08,Colorado,Sun,Colorado Roster


In [366]:
fulldf.head(60)

,#,Name,G,A,+/-,PIM,S,SOFF,LB,T,CT,FO,TOF,Date,Location,Day,Team
0,0,T.Welsh,2,0,0,4,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
1,0,P.Welsh,0,0,0,2,0,0,5,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
2,0,J.Tucker,2,0,0,2,0,0,6,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
3,0,R.Sowell,0,0,0,0,0,0,0,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
4,0,D.Smith,0,0,0,0,0,0,3,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
5,0,T.Millon,1,0,0,2,0,0,2,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
6,0,B.Marino,1,1,0,0,0,0,1,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
7,0,T.Manos,0,0,0,0,0,0,0,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
8,0,B.Kroneberger,1,0,0,0,0,0,2,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster
9,0,M.Kerwick,1,0,0,0,0,0,0,0,0,-,,1993-03-28,New York,Sun,Baltimore Roster


In [368]:
fulldf.to_csv('NLLFloorPlayersGameStats.csv')